In [15]:
import pandas as pd
import numpy as np
import seaborn as sns

df = pd.read_csv("/Users/ipatel/Documents/Amit/Python/Practice/p1/notbook/raw.csv")
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [16]:
df.shape

(891, 8)

In [17]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [18]:
df['Family'] = df['SibSp'] + df['Parch']

In [22]:

X = df.drop(columns=['Survived', 'SibSp', 'Parch'], axis=1)
y = df['Survived']

y.sample(5)

50     0
459    0
212    0
68     1
399    1
Name: Survived, dtype: int64

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

num_columns = X_train.select_dtypes(include=np.number).columns.tolist()
cat_columns = X_train.select_dtypes(exclude=np.number).columns.tolist()
print(f'cat_columns {cat_columns}')
print(f'num_columns {num_columns}')


# Define Transformers Numerical and Categorical Columns
imput_numeric_col = SimpleImputer(strategy="mean")
num_scaler = StandardScaler()
cat_scaler = StandardScaler(with_mean=False)
imput_category_col = SimpleImputer(strategy="most_frequent")
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Combined Column Transformer for categorical and numerical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num_col', Pipeline(steps=[
            ('impute', imput_numeric_col),
            ('scale', num_scaler)
        ]), num_columns),
        ('cat_col', Pipeline(steps=[
            ('impute', imput_category_col),            
            ('onehot', ohe),
            ('scaler', cat_scaler),
        ]), cat_columns)
    ]
)

# Create Pipeline
pip = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LinearRegression())
])

# Fit Pipeline
pip.fit(X_train, y_train)


cat_columns ['Sex', 'Embarked']
num_columns ['Pclass', 'Age', 'Fare', 'Family']


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_col',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  ['Pclass', 'Age', 'Fare',
                                                   'Family']),
                                                 ('cat_col',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False)),
                                                                  ('scaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  ['Sex', 'Embarked'])])),
                ('model', LinearRegression())])

In [21]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict
pred = pip.predict(X_test)

# Calculate and print accuracy metrics
mse = mean_squared_error(y_test, pred)
r2 = r2_score(y_test, pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 0.135395440294604
R^2 Score: 0.44167241924331946
